In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, date
from io import BytesIO
from ipywidgets import (
    FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox,
    FloatSlider, Dropdown, Text
)
from IPython.display import display, clear_output
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import copy

output_all = Output()
df_global = None
selected_assets = []
sector_mapper = {}
sector_constraint_widgets = {}
expected_return_inputs = {}
constraint_rows = []
constraint_inputs = []

upload = FileUpload(accept=".xlsx", multiple=False)

def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

# Constraint binding helpers
def make_equal_constraint(i, v): return lambda x: x[i] == v
def make_le_constraint(i, v): return lambda x: x[i] <= v
def make_ge_constraint(i, v): return lambda x: x[i] >= v
def make_equal_pair_constraint(i, j): return lambda x: x[i] == x[j]
def make_le_pair_constraint(i, j): return lambda x: x[i] <= x[j]
def make_ge_pair_constraint(i, j): return lambda x: x[i] >= x[j]

def handle_upload(change):
    global df_global
    with output_all:
        clear_output()
        try:
            uploaded_filename = list(upload.value.keys())[0]
            content = upload.value[uploaded_filename]["content"]
            df = pd.read_excel(BytesIO(content), sheet_name="PyData", engine="openpyxl")
            df = df.rename(columns=df.iloc[0]).drop(0)
            df["Dates"] = pd.to_datetime(df["Dates"])
            df = df.set_index("Dates")
            df = df.astype(float)
            df = df[df.index > datetime(2017, 1, 1)]
            df_global = df
            print("✅ File loaded successfully.")
            show_asset_selector()
        except Exception as e:
            print(f"❌ Error loading file: {e}")

def show_asset_selector():
    with output_all:
        clear_output()
        if df_global is None:
            print("⚠️ No data found. Please upload a valid Excel file.")
            return

        asset_select = SelectMultiple(description="Assets:", options=df_global.columns.tolist(), rows=10)
        continue_btn = Button(description="➡ Define Sectors", button_style="primary")

        def proceed(change):
            global selected_assets
            selected_assets = list(asset_select.value)
            if not selected_assets:
                print("⚠️ Please select at least one asset.")
                return
            show_expected_return_inputs()

        continue_btn.on_click(proceed)
        display(VBox([
            Label("📊 Step 2: Select assets:"),
            asset_select,
            continue_btn
        ]))

upload.observe(handle_upload, names="value")

display(VBox([
    Label("📥 Step 1: Upload Excel file (sheet: PyData):"),
    upload,
    output_all
]))
